In [13]:
#adaptive rewiring using monte carlo method. 
#each simulation step only one node is chosen and the state of that node is updated
#simulation runs by choosing nodes and not the edges

In [14]:
#importing the required libraries

import networkx as nx #network library to generate inital networks and perform various network operations
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# import cProfile #for timing the loops
# from numba import jit #numba precomplier to make the code faster
from mpl_toolkits.mplot3d import Axes3D
from copy import deepcopy
import sys
import pandas as pd

In [15]:
# #opening a file to store data
# f = open("adap_rew.txt","w+") 

In [16]:
N = 100  #number of individuals
T = 10000 #number of time steps

In [17]:
timeseries = pd.DataFrame(index=range(T+1),columns=range(216))

In [18]:
#making the nodes interact

# @jit(nopython=True)
def interaction(N,k,indi_state,p,counter) :
    #making current state of all nodes the same as the previous state
    indi_state[:,k] = indi_state[:,k-1]
    
    #changing just one element of the current indi_state since only one node reacts at every simulation step
    suscep_node_list = susceptible_node_list(G,k,indi_state)
    n = np.random.randint(N) #choosing a random node
#     print("n first chosen =",n)
    H = deepcopy(G)
    connec_neigh_list = list(G.neighbors(n))
    if (len(connec_neigh_list) == 0) :
        print('n=',n)
        print('oops! This node is not connected to any other node!!')
        nx.draw_networkx(G,pos=nx.circular_layout(G),with_labels=False,nodecolor='r',edge_color='b')
        new_n = n
        while new_n == n and (len(list(G.neighbors(new_n))) == 0):
            new_n  = np.random.randint(N) #choosing a new random node
            print('new_n=',n)
            n = new_n
#         sys.exit()
    
    connec_neigh_list = list(G.neighbors(n))
    partner = random.choice(connec_neigh_list) #choosing a random connected neighbour of the node
    
    z11 = np.random.uniform() #choosing either rewiring or interaction
    z12 = np.random.uniform() #if interaction, then susceptible becoming infected 
    z13 = np.random.uniform() #if interaction, then infected becoming susceptible 

    #if one is infected and the other is susceptible and they are connected 
    #SI link
    if ((indi_state[n,k-1] == 1) and (indi_state[partner,k-1] == 2)) : #if the chosen node is susceptible and is linked to infected
        if (w >= z11) and (len(suscep_node_list)!=0) : 
            #special case when rewiring is not feasible
            if (len(suscep_node_list) == 1) and (suscep_node_list[0] == n) : #if there is only one susceptible node and it is the same node (to prevent rewiring to itself)
                int_keeping_si(indi_state,n,k,z12,p)
            else : #rewiring
                H = deepcopy(new_link(G,indi_state,n,k,suscep_node_list,partner))
                counter = counter + 1
        else : #no change in network. Continue keeping the same G
            int_keeping_si(indi_state,n,k,z12,p)
    #II link               
    #if both are infected and they are connected
    elif ((indi_state[n,k-1] == 2) and (indi_state[partner,k-1] == 2)) :
        ii_link_interaction(r,z13,indi_state,n,k)

    #neither SI nor II
    else :
        #no interaction at all'
        indi_state[n,k] = indi_state[n,k-1]
    
#all others retain their previous state
#this is implemented in the line commented "#making current state of all nodes the same as the previous state"

    return indi_state,counter,H

In [19]:
#creating a list of nodes of the current graph which are susceptible after the previous time step

def susceptible_node_list(G,k,indi_state) :
    G_temp = deepcopy(G)
    suscep_node_list = []
    for i in G_temp.nodes() :
        if indi_state[i,k-1] == 1 :
            suscep_node_list.extend([i])
#         else :
#             print('no more susceptible nodes exist to link to')
    
    return suscep_node_list

In [20]:
def new_link(G,indi_state,n,k,suscep_node_list,partner) :
    G_temp = deepcopy(G)
#breaking SI and making a new link with some other random S and also giving a chance for the I to recover

    G_temp.remove_edge(n,partner)
    #identify some other random susceptible node
    rand_suscep_node = random.choice(suscep_node_list)
    while (rand_suscep_node == n) : #making sure that the susceptible node does not re-link to itself
        rand_suscep_node = random.choice(suscep_node_list)
    G_temp.add_edge(n,rand_suscep_node)

        
    return G_temp

In [21]:
def int_keeping_si(indi_state,n,k,z12,p) :
#interaction keeping SI
    if p >= z12 :
        #susceptible becomes infected 
        indi_state[n,k] = 2 #susceptible becomes infected
    else :
        indi_state[n,k] = 1 #susceptible remains susceptible
        
    return indi_state

In [22]:
def ii_link_interaction(r,z13,indi_state,n,k) : 
    if r >= z13 : #infected becomes susceptible
        indi_state[n,k] = 1
    else :
        indi_state[n,k] = 2 #infected remains infected
        
    return indi_state

In [23]:
#creating the static Watts-Strogatz network
def static_network(N,nn,p_rew) :
    G = nx.watts_strogatz_graph(N,nn,p_rew) 
    return G   

In [24]:
#user controlled variables
dataframe_index_2 = 0
G = nx.watts_strogatz_graph(10,5,0) 
H = deepcopy(G)

for p in range(0,12,2) : #susceptible going to infected
    p = p*0.1
#     p = 1
    for start in range(0,12,2) : #initial condition loop
        start = start*0.1
#         start = 1
        for r in range(5,6) : #infected recovering to susceptible
            r = r*0.1
            for w in range(0,12,2) : #probability of SI link rewiring to SS
                w = w*0.1
                
                del G
                del H
                
                print("p=",p)
                print("start=",start)
                print("r=",r)
                print("w=",w)
                
                p_rew = 1
                nn = 6 #each node is connected to nn number of nearest neighbours, nn-1 neighbors if nn is odd

#                 N = 100  #number of individuals
#                 T = 100 #number of time steps

                phi = 999*np.ones([1,T]) #fraction of steady state infected

                t = np.arange(0,T) #time steps
                t = np.asarray(t).reshape(1,T)

    #             indi_state = np.zeros((N,T),dtype=np.int32) #stores the state of each individual
                indi_state = np.zeros((N,2),dtype=np.int32) #stores the state of each individual
                indi_state[:,0] = 1


                #initial state of individuals
                infec = random.sample(range(0, N), np.int32(start*N)) #radnomly infecting few individuals as initial condition
                # np.random.shuffle(infec)
                infec = np.asarray(infec).reshape(np.int32(start*N),1)
                if start != 0 :
                    indi_state[infec[:,0],0] = 2

                phi[0,0] = np.count_nonzero(indi_state[:,0] == 2)/N
                phi_star = phi[0,0]
                timeseries.iloc[0,0] = phi[0,0]
                print("phi_star[0,0]=",phi_star)

                #creating the static network
                G = static_network(N,nn,p_rew) #calling the function to create the static network
    #             nx.draw_networkx(G,pos=nx.circular_layout(G),with_labels=True,nodecolor='r',edge_color='b')
    
                dataframe_index_1 = 1
                for sim in range(0,T): #loop for simulation step
                    if (sim%1000 == 0) :
                        print('sim=',sim)
                    k = 1
    #                 print('k = ',k)
                    counter = 0 #to check whether rewiring has taken place
                    indi_state, counter, H = interaction(N,k,indi_state,p,counter) #calling the interaction function
                    phi[0,k] = np.count_nonzero(indi_state[:,k] == 2)/N

    #                 f.write("%f\t%f\n" % (k,phi[0,k]))
    #                 f.flush()
                    if counter != 0 : #rewiring has taken place
                        G = deepcopy(H) #replacing the exisitng graph with a new grap
                    timeseries.iloc[dataframe_index_1,dataframe_index_2] = phi[0,k]
                    dataframe_index_1 = dataframe_index_1 + 1

                    #after all the calculation of the current simulation step,
                    #copying the elements of the current indi_state to the previous indi_state
                    #in order to make the array ready for the next simulation
                    indi_state[:,k-1] = indi_state[:,k]
                    
                dataframe_index_2 = dataframe_index_2 + 1

p= 0.0
start= 0.0
r= 0.5
w= 0.0
phi_star[0,0]= 0.0
sim= 0
sim= 1000
sim= 2000
sim= 3000
sim= 4000
sim= 5000
sim= 6000
sim= 7000
sim= 8000
sim= 9000
p= 0.0
start= 0.0
r= 0.5
w= 0.2
phi_star[0,0]= 0.0
sim= 0
sim= 1000
sim= 2000
sim= 3000
sim= 4000
sim= 5000
sim= 6000
sim= 7000
sim= 8000
sim= 9000
p= 0.0
start= 0.0
r= 0.5
w= 0.4
phi_star[0,0]= 0.0
sim= 0
sim= 1000
sim= 2000
sim= 3000
sim= 4000
sim= 5000
sim= 6000
sim= 7000
sim= 8000


KeyboardInterrupt: 

In [ ]:
timeseries

In [ ]:
# %timeit interaction(N,k,adj_mat,indi_state,p,counter)
# # %timeit new_link(G,indi_state,n,k)
# # %timeit int_keeping_si(indi_state,partner,n,k,z12,z13)
# # %timeit ii_link_interaction(r,z13,indi_state,partner,n,k)

In [ ]:
# # '''time series plot'''


# number = 36
# for i in range(36+3,36+36,6) :
#     nr=(i)%number
#     a = np.loadtxt("adap_rew.txt")[i,:]
#     plt.plot(a,label='plot nr %i'%(int(i/number)+1)+"point %i"%nr)
#     plt.legend(bbox_to_anchor=(1,1))

In [ ]:
# import fileinput
# with fileinput.input(files=('adap_rew.txt')) as f:
#     i = 1
#     for line in f:
#         a = line
#         break

In [ ]:
# print(a[6])

In [ ]:
# import fileinput

# '''averaging time series data'''
# f = open("adap_rew_phi_star_data_ws.txt","w+") 
# k = T
# # phi_star = sum(a[counter,k-100:k])/100 #obtaining average phi_star of the last 100 time steps
# # print("phi_star=",phi_star)

# counter = 0
# for p in range(0,12,2) : #susceptible going to infected
#     p = p*0.1
# #     p = 1
#     for start in range(0,12,2) : #initial condition loop
#         start = start*0.1
# #         start = 1
#         for r in range(5,6) : #infected recovering to susceptible
#             r = r*0.1
#             for w in range(0,12,2) : #probability of SI link rewiring to SS
#                 w = w*0.1
# #                 a = np.genfromtxt("adap_rew.txt", max_rows=1)
#                 with fileinput.input(files=('adap_rew.txt')) as f:
#                 phi_star = np.average(a)
#     #             print(phi_star)
#                 f.write("%f\t%f\t%f\t%f\t%f\n" % (p,start,r,w,phi_star))
#                 f.flush()
#                 counter = counter + 1
# print(counter)

In [ ]:
# # a = np.loadtxt("epid_p_phiini_prew_phi_star_data.txt")[:,0]
# # b =  np.loadtxt("epid_p_phiini_prew_phi_star_data.txt")[:,1]
# # c =  np.loadtxt("epid_p_phiini_prew_phi_star_data.txt")[:,2]
# d = np.loadtxt("adap_rew_1.txt")[:,3]


# counter = 0
# l = 0
# fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(14,14))
# i = 0
# for row in ax:
#     k = 0
#     for col in row:
#         print(col)
#         ax[i, k].set_title('p= %f' %l)
#         l = l + 0.2
#         j = 36*counter
#         im = col.scatter(np.loadtxt("adap_rew_1.txt")[j:36*(counter+1),1],np.loadtxt("adap_rew_1.txt")[j:36*(counter+1),3], c=np.loadtxt("adap_rew_1.txt")[j:36*(counter+1),4],s=200,vmin=0, vmax=1)
#         k = k + 1
#         counter = counter + 1
#     i = i + 1

# nax = fig.add_subplot(111, frame_on = False)
# nax.set_xticks([])
# nax.set_yticks([])
# fig.suptitle(r'$\phi^*$, adaptive rewiring. $nn=6,N=100,N_t=10000,r=0.5$',fontsize = 20)
# nax.set_xlabel(r'$\phi_{ini}$', fontsize = 20, labelpad=20)
# nax.set_ylabel(r'$p_{rew}$', fontsize = 20, labelpad=40)
# fig.subplots_adjust(right=0.8)

# cbar_ax = fig.add_axes([0.85, 0.15, 0.015, 0.7])
# fig.colorbar(im, cax=cbar_ax)

# plt.savefig("test.svg", format="svg")

# # for i in range(1, 13):
# #     plt.subplot(4, 3, i)
# #     j = 121*counter
# #     print(j)
# #     plt.scatter(np.loadtxt("epid_p_phiini_prew_phi_star_data.txt")[j:121*(counter+1),1],np.loadtxt("epid_p_phiini_prew_phi_star_data.txt")[j:121*(counter+1),2], c=np.loadtxt("epid_p_phiini_prew_phi_star_data.txt")[j:121*(counter+1),3],s=200)
# # #     plt.colorbar()
# #     # plt.title(r'$\phi^*$, Critical Threshold Model. $d^*=3,T=12, N=100,N_t=1000,r=1,rho=1$',fontsize=15,fontweight='bold')
# #     # plt.title(r'$\phi^*$, Epidemic Threshold Model. $d^*=1,T=2, N=100,N_t=1000,r=0.5,rho=1$',fontsize=15,fontweight='bold')
# #     plt.title('p')
# #     plt.xlabel(r'$\phi_{ini}$',fontsize=15)
# #     plt.ylabel(r'$\phi_{rew}$',fontsize=15)
# #     counter = counter + 1 
